In [8]:
import networkx as nx
import datasets

# add .. to pythonpath
import sys
sys.path.append('..')
from dataset_loaders import *
from utils import eg2nx

In [10]:
# cd to ..
import os
os.chdir('/Users/tscp/testdir/easygraph-bench/')
chem = eg2nx(load_cheminformatics())
chem

################################################################################
loading graph cheminformatics ...
################################################################################
################################################################################
finished loading graph cheminformatics
ENZYMES_g1: nodes: 37 edges: 168 is_directed: True
################################################################################


In [8]:
from itertools import islice
list(islice(nx.generate_edgelist(chem), 10))

['2 1 {}',
 '2 3 {}',
 '2 4 {}',
 '2 25 {}',
 '2 28 {}',
 '1 2 {}',
 '1 3 {}',
 '1 4 {}',
 '3 1 {}',
 '3 2 {}']

In [14]:
# load https://huggingface.co/datasets/easygraph-bench/cheminformatics from huggingface hub
# c = 
c = datasets.load_dataset('easygraph-bench/cheminformatics')
# c

FileNotFoundError: Couldn't find a dataset script at /Users/tscp/testdir/easygraph-bench/easygraph-bench/cheminformatics/cheminformatics.py or any data file in the same directory. Couldn't find 'easygraph-bench/cheminformatics' on the Hugging Face Hub either: FileNotFoundError: Unable to resolve any data file that matches ['**'] in dataset repository easygraph-bench/cheminformatics with any supported extension ['csv', 'tsv', 'json', 'jsonl', 'parquet', 'txt', 'blp', 'bmp', 'dib', 'bufr', 'cur', 'pcx', 'dcx', 'dds', 'ps', 'eps', 'fit', 'fits', 'fli', 'flc', 'ftc', 'ftu', 'gbr', 'gif', 'grib', 'h5', 'hdf', 'png', 'apng', 'jp2', 'j2k', 'jpc', 'jpf', 'jpx', 'j2c', 'icns', 'ico', 'im', 'iim', 'tif', 'tiff', 'jfif', 'jpe', 'jpg', 'jpeg', 'mpg', 'mpeg', 'msp', 'pcd', 'pxr', 'pbm', 'pgm', 'ppm', 'pnm', 'psd', 'bw', 'rgb', 'rgba', 'sgi', 'ras', 'tga', 'icb', 'vda', 'vst', 'webp', 'wmf', 'emf', 'xbm', 'xpm', 'BLP', 'BMP', 'DIB', 'BUFR', 'CUR', 'PCX', 'DCX', 'DDS', 'PS', 'EPS', 'FIT', 'FITS', 'FL

In [ ]:
datasets.list_datasets()

In [13]:
len(_)

25647

In [16]:
chem.edges[('2', '1')]

{}

In [42]:
import json
import networkx as nx

def convert_graph_to_jsonl(graph: nx.Graph | nx.DiGraph, filename):
    is_directed = isinstance(graph, nx.DiGraph)
    with open(filename, 'w') as f:
        f.write(json.dumps({'directed': is_directed}) + '\n')
        for node in graph.nodes:
            data = {'id': node}
            for key, value in graph.nodes[node].items():
                data[key] = value
            f.write(json.dumps(data) + '\n')
        for edge in graph.edges:
            data = {'source': edge[0], 'target': edge[1]}
            for key, value in graph.edges[edge].items():
                data[key] = value
            f.write(json.dumps(data) + '\n')

def create_graph_from_jsonl(filename, directed: bool | None = None):
    def get_is_directed() -> bool | None:
        # read the first line to determine if the graph is directed
        is_directed = None
        with open(filename, 'r') as f:
            line = f.readline()
            data = json.loads(line.strip())
            if 'directed' in data:
                is_directed = data['directed']
        return is_directed
    # check if directed is set, if not set, call get_is_directed to get is_directed, if is_directed is None, defaults to False
    if directed is None:
        directed = get_is_directed()
        if directed is None:
            directed = False

    if directed:
        graph = nx.DiGraph()
    else:
        graph = nx.Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            if 'id' in data:
                id = data.pop('id')
                graph.add_node(id, **data)
            elif 'source' in data and 'target' in data:
                source = data.pop('source')
                target = data.pop('target')
                graph.add_edge(source, target, **data)
    return graph



In [ ]:
from typing import Dict
import networkx as nx
from datasets import Dataset, Features, Value, ClassLabel

def load_dataset(filename: str, directed: bool = False) -> Dict[str, Dataset]:
    # Define the features for the dataset
    features = Features({
        'graph': {'nodes': Features({'attr': Dict([Value('string')])}),
                  'edges': Features({'source': Value('int64'),
                                     'target': Value('int64'),
                                     'attr': Dict(Value('string'))})}
    })

    # Load the JSONL file into a NetworkX graph
    graph = create_graph_from_jsonl(filename, directed=directed)

    # Convert the graph to a dictionary format that matches the features
    nodes = [{'id': node, 'attr': attrs} for node, attrs in graph.nodes(data=True)]
    edges = [{'source': u, 'target': v, 'attr': attrs} for u, v, attrs in graph.edges(data=True)]
    data = {'graph': {'nodes': nodes, 'edges': edges}}

    # Create a dataset from the dictionary data using the defined features
    dataset = Dataset.from_dict(data, features=features)

    # Split the dataset into train, validation, and test sets
    train_dataset = dataset.shuffle().select(range(100))
    valid_dataset = dataset.shuffle().select(range(100, 150))
    test_dataset = dataset.shuffle().select(range(150, 200))

    # Return a DatasetDict object with the splits
    return {'train': train_dataset, 'validation': valid_dataset, 'test': test_dataset}


In [48]:
convert_graph_to_jsonl(chem, 'chem.jsonl')
chem2 = create_graph_from_jsonl('chem.jsonl')
chem2

In [44]:
# check if chem and chem2 are the same
from networkx.utils import nodes_equal, edges_equal
nodes_equal(chem, chem2) and edges_equal(chem, chem2)

IndexError: string index out of range

In [50]:
len(chem.nodes), len(chem.edges), len(chem2.nodes), len(chem2.edges)
chem.nodes, chem2.nodes
chem.edges, chem2.edges
# chem.nodes['1'], chem2.nodes['1']
# chem.edges[('2', '1')], chem2.edges[('2', '1')]

(OutEdgeView([('2', '1'), ('2', '3'), ('2', '4'), ('2', '25'), ('2', '28'), ('1', '2'), ('1', '3'), ('1', '4'), ('3', '1'), ('3', '2'), ('3', '4'), ('3', '28'), ('3', '29'), ('4', '1'), ('4', '2'), ('4', '3'), ('4', '5'), ('4', '6'), ('4', '29'), ('25', '2'), ('25', '28'), ('25', '29'), ('25', '30'), ('28', '2'), ('28', '3'), ('28', '25'), ('28', '29'), ('28', '30'), ('29', '3'), ('29', '4'), ('29', '25'), ('29', '27'), ('29', '28'), ('29', '30'), ('5', '4'), ('5', '6'), ('5', '7'), ('5', '30'), ('6', '4'), ('6', '5'), ('6', '7'), ('6', '8'), ('6', '30'), ('30', '5'), ('30', '6'), ('30', '25'), ('30', '26'), ('30', '27'), ('30', '28'), ('30', '29'), ('27', '12'), ('27', '13'), ('27', '26'), ('27', '29'), ('27', '30'), ('7', '5'), ('7', '6'), ('7', '8'), ('7', '9'), ('8', '6'), ('8', '7'), ('8', '9'), ('8', '10'), ('8', '11'), ('9', '7'), ('9', '8'), ('9', '10'), ('26', '13'), ('26', '14'), ('26', '15'), ('26', '27'), ('26', '30'), ('10', '8'), ('10', '9'), ('10', '11'), ('10', '12'), (

In [3]:
from datasets import load_dataset
s = squad = load_dataset('squad')
s

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /Users/tscp/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
import datasets
datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": "/Users/tscp/testdir/test/chem.tsv"}),

(SplitGenerator(name='train', gen_kwargs={'filepath': '/Users/tscp/testdir/test/chem.tsv'}, split_info=SplitInfo(name='train', num_bytes=0, num_examples=0, shard_lengths=None, dataset_name=None)),)

In [5]:
sg = _
list(sg())

TypeError: 'tuple' object is not callable

In [12]:
from datasets import Dataset
ds = Dataset.from_generator(nx.generate_edgelist(chem))

TypeError: cannot pickle 'generator' object

In [14]:
from datasets import Dataset
def gen():
    yield 'a b'
    yield {"pokemon": "bulbasaur", "type": "grass"}
    yield {"pokemon": "squirtle", "type": "water"}
ds = Dataset.from_generator(gen)
ds[0]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [16]:
dataset = load_dataset('json', '/Users/tscp/testdir/easygraph-bench/chem.jsonl')

InvalidConfigName: Bad characters from black list '<>:/\|?*' found in '/Users/tscp/testdir/easygraph-bench/chem.jsonl'. They could create issues when creating a directory for this config on Windows filesystem.